In [5]:
import math
import numpy as np
import pandas as pd
import pickle
import re
import string
import random
import logging
import os

from gensim.models import Word2Vec

from nltk.tokenize import sent_tokenize
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer as CV

from langdetect import detect


SEED = 42
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)

In [3]:
df = pd.read_parquet("../data/20220602 - Application full text.parquet",
                     engine="fastparquet",
#                      header=0, 
#                      dtype={'Application_id': str,})
#                      "clean_text": str}
                    )
df = df[["Application_id", "Project_description_clean"]]
df = df.dropna()
# df = df.fillna(0)
# df["text"] = df["clean_text"]
df["text"] = df["Project_description_clean"]
# custom_stopwords = set(stopwords.words("english"))

dat = pd.read_csv("../data/reg_data_R.csv", dtype={'Application_id': str,})
dat_df = df.merge(dat, on="Application_id")

<ipython-input-3-0510137ad2ba>:14: DtypeWarning: Columns (21,41,45,52) have mixed types. Specify dtype option on import or set low_memory=False.
  dat = pd.read_csv("../data/reg_data_R.csv", dtype={'Application_id': str,})


In [7]:
def clean_text(text, tokenizer):
    """Pre-process text and generate tokens

    Args:
        text: Text to tokenize.

    Returns:
        Tokenized text.
    """
    text = str(text).lower()  # Lowercase words
#     text = re.sub(r"\[(.*?)\]", "", text)  # Remove [+XYZ chars] in content
#     text = re.sub(r"\s+", " ", text)  # Remove multiple spaces in content
#     text = re.sub(r"\w+|", "", text)  # Remove ellipsis (and last word)
#     text = re.sub(r"(?<=\w)-(?=\w)", " ", text)  # Replace dash between words
    text = re.sub(
        f"[{re.escape(string.punctuation)}]", "", text
    )  # Remove punctuation
    
#     text = re.sub("\d+", "", text) # remove digits

    tokens = tokenizer(text)  # Get tokens from text
#     tokens = [t for t in tokens if not t in stopwords]  # Remove stopwords
#     tokens = ["" if t.isdigit() else t for t in tokens]  # Remove digits
    tokens = [t for t in tokens if len(t) > 1]  # Remove short tokens
    return tokens

In [9]:
dat_df["text"] = dat_df["text"].fillna("")

# dat_df["english"] = dat_df["text"].map(lambda x:detect(x)=="en")

dat_df["tokens"] = dat_df["text"].map(lambda x: clean_text(x, word_tokenize))

In [10]:
hype_dict = {
'Importance': ['compelling',
 'critical',
 'crucial',
 'essential',
 'foundational',
 'fundamental',
 'imperative',
 'important',
 'indispensable',
 'invaluable',
 'key',
 'major',
 'paramount',
 'pivotal',
 'significant',
 'strategic',
 'timely',
 'ultimate',
 'urgent',
 'vital'],

'Novelty': ['creative',
 'emerging',
 'first',
 'groundbreaking',
 'innovative',
 'latest',
 'novel',
 'revolutionary',
 'unique',
 'unparalleled',
 'unprecedented'],
             
'Rigor': ['accurate',
 'advanced',
 'careful',
 'cohesive',
 'detailed',
 'nuanced',
 'powerful',
 'quality',
 'reproducible',
 'rigorous',
 'robust',
 'scientific',
 'sophisticated',
 'strong',
 'systematic'],
             
'Scale': ['ample',
 'biggest',
 'broad',
 'comprehensive',
 'considerable',
 'deeper',
 'diverse',
 'enormous',
 'expansive',
 'extensive',
 'fastest',
 'greatest',
 'huge',
 'immediate',
 'immense',
 'interdisciplinary',
 'international',
 'interprofessional',
 'largest',
 'massive',
 'multidisciplinary',
 'myriad',
 'overwhelming',
 'substantial',
 'top',
 'transdisciplinary',
 'tremendous',
 'vast'],
             
'Utility': ['accessible',
 'actionable',
 'deployable',
 'durable',
 'easy',
 'effective',
 'efficacious',
 'efficient',
 'generalizable',
 'ideal',
 'impactful',
 'intuitive',
 'meaningful',
 'productive',
 'ready',
 'relevant',
 'rich',
 'safer',
 'scalable',
 'seamless',
 'sustainable',
 'synergistic',
 'tailored',
 'tangible',
 'transformative',
 'userfriendly'],
             
'Quality': ['ambitious',
 'collegial',
 'dedicated',
 'exceptional',
 'experienced',
 'intellectual',
 'longstanding',
 'motivated',
 'premier',
 'prestigious',
 'promising',
 'qualified',
 'renowned',
 'senior',
 'skilled',
 'stellar',
 'successful',
 'talented',
 'vibrant'],
             
'Attitude': ['attractive',
 'confident',
 'exciting',
 'incredible',
 'interesting',
 'intriguing',
 'notable',
 'outstanding',
 'remarkable',
 'surprising'],
             
'Problem': ['alarming',
 'daunting',
 'desperate',
 'devastating',
 'dire',
 'dismal',
 'elusive',
 'stark',
 'unanswered',
 'unmet']
}

hype_list = list(set([j for l in hype_dict.values() for j in l]))

In [ ]:
# get hype words' synonyms
hype_syns = []

hype_and_syns = set(hype_list).union(set(hype_syns))

In [ ]:
for i in range(100):
    dat_df[i] = 0
    for ind, row in dat_df.iterrows():
        # randomly replace hype words
        rand_new_text = [t if t not in hype_syns else random.sample(hype_and_syns, 1)
                            for t in row["tokens"]]
        # calculate new distribution
        dat_df.loc[ind, i] = sum([1 if t in hype_list # or hype_and_syns?
                                      else 0
                                      for t in rand_new_text])